In [ ]:
import os

from llama_stack_client import LlamaStackClient, Agent, AgentEventLogger
from termcolor import colored
from dotenv import load_dotenv
from pydantic import BaseModel

import json
from rich.pretty import pprint
from uuid import uuid4

In [ ]:
load_dotenv()

In [ ]:
host = os.environ["TOGETHER_URL"]
together_api_key = os.environ["TOGETHER_API_KEY"]

client = LlamaStackClient(
    base_url=host,
    provider_data={
        "tavily_search_api_key": os.getenv("TAVILY_SEARCH_API_KEY"),
        "together_api_key": together_api_key,
    },
)

In [ ]:
from llama_stack_client import Agent, AgentEventLogger

agent = Agent(
    client, 
    model="meta-llama/Llama-3.3-70B-Instruct",
    instructions="You are a helpful assistant. Use web_search tool to answer the questions.",
    tools=["builtin::websearch"],
)
user_prompts = [
    "Which teams played in the NBA western conference finals of 2024. Search the web for the answer.",
    "In which episode and season of South Park does Bill Cosby (BSM-471) first appear? Give me the number and title. Search the web for the answer.",
    "What is the British-American kickboxer Andrew Tate's kickboxing name? Search the web for the answer.",
]

session_id = agent.create_session(uuid4().hex)

for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )

    for log in AgentEventLogger().log(response):
        log.print()


In [ ]:
from rich.pretty import pprint

session_response = client.agents.session.retrieve(
    session_id=session_id,
    agent_id=agent.agent_id,
)

pprint(session_response.turns)

In [ ]:
num_tool_call = 0
for turn in session_response.turns:
    for step in turn.steps:
        if step.step_type == "tool_execution" and step.tool_calls[0].tool_name == "brave_search":
            num_tool_call += 1

print(f"{num_tool_call}/{len(session_response.turns)} user prompts are followed by a tool call to `brave_search`")

In [ ]:
eval_rows = []

expected_answers = [
    "Dallas Mavericks and the Minnesota Timberwolves",
    "Season 4, Episode 12",
    "King Cobra",
]

for i, turn in enumerate(session_response.turns):
    eval_rows.append(
        {
            "input_query": turn.input_messages[0].content,
            "generated_answer": turn.output_message.content,
            "expected_answer": expected_answers[i],
        }
    )

pprint(eval_rows)

scoring_params = {
    "basic::subset_of": None,
}
scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)
pprint(scoring_response)

In [ ]:
print(f"Getting traces for session_id={session_id}")
import json

from rich.pretty import pprint

agent_logs = []

for span in client.telemetry.query_spans(
    attribute_filters=[
        {"key": "session_id", "op": "eq", "value": session_id},
    ],
    attributes_to_return=["input", "output"],
):
    if span.attributes["output"] != "no shields":
        agent_logs.append(span.attributes)

print("Here are examples of traces:")
pprint(agent_logs[:2])


In [ ]:
import ast
import json

eval_rows = []

for log in agent_logs:
    input = json.loads(log["input"])
    if isinstance(input, list):
        input = input[-1]
    if input["role"] == "user":
        eval_rows.append(
            {
                "input_query": input["content"],
                "generated_answer":  log["output"],
                # check if generated_answer uses tools brave_search
                "expected_answer": "brave_search",
            },
        )

pprint(eval_rows)

scoring_params = {
    "basic::subset_of": None,
}

scoring_response = client.scoring.score(
    input_rows=eval_rows, scoring_functions=scoring_params
)

pprint(scoring_response)


In [ ]:
import rich
from rich.pretty import pprint

judge_model_id = "meta-llama/Llama-3.3-70B-Instruct"

JUDGE_PROMPT = """
Given a QUESTION and GENERATED_RESPONSE and EXPECTED_RESPONSE.

Compare the factual content of the GENERATED_RESPONSE with the EXPECTED_RESPONSE. Ignore any differences in style, grammar, or punctuation.
  The GENERATED_RESPONSE may either be a subset or superset of the EXPECTED_RESPONSE, or it may conflict with it. Determine which case applies. Answer the question by selecting one of the following options:
  (A) The GENERATED_RESPONSE is a subset of the EXPECTED_RESPONSE and is fully consistent with it.
  (B) The GENERATED_RESPONSE is a superset of the EXPECTED_RESPONSE and is fully consistent with it.
  (C) The GENERATED_RESPONSE contains all the same details as the EXPECTED_RESPONSE.
  (D) There is a disagreement between the GENERATED_RESPONSE and the EXPECTED_RESPONSE.
  (E) The answers differ, but these differences don't matter from the perspective of factuality.

Give your answer in the format "Answer: One of ABCDE, Explanation: ".

Your actual task:

QUESTION: {input_query}
GENERATED_RESPONSE: {generated_answer}
EXPECTED_RESPONSE: {expected_answer}
"""

input_query = (
    "What are the top 5 topics that were explained? Only list succinct bullet points."
)
generated_answer = """
Here are the top 5 topics that were explained in the documentation for Torchtune:

* What is LoRA and how does it work?
* Fine-tuning with LoRA: memory savings and parameter-efficient finetuning
* Running a LoRA finetune with Torchtune: overview and recipe
* Experimenting with different LoRA configurations: rank, alpha, and attention modules
* LoRA finetuning
"""
expected_answer = """LoRA"""

rows = [
    {
        "input_query": input_query,
        "generated_answer": generated_answer,
        "expected_answer": expected_answer,
    },
]

scoring_params = {
    "llm-as-judge::base": {
        "judge_model": judge_model_id,
        "prompt_template": JUDGE_PROMPT,
        "type": "llm_as_judge",
        "judge_score_regexes": ["Answer: (A|B|C|D|E)"],
    },
    "basic::subset_of": None,
}

response = client.scoring.score(
    input_rows=rows, 
    scoring_functions=scoring_params)

pprint(response)
